# Degree Preserving Model Comparison

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import pandas as pd
import csv

In [2]:
G = nx.read_edgelist('../networkx_df.csv', comments='#',
                     create_using=nx.DiGraph(), 
                     delimiter=',', 
                     nodetype=int, 
                     encoding='utf-8')

In [4]:


[
    len(c)
    for c in sorted(nx.weakly_connected_components(G), key=len, reverse=True)
]

[8932, 10, 9, 8, 7, 4, 3, 3, 3]

In [5]:
all_components = nx.weakly_connected_components(G)

In [8]:
Gcc = sorted(nx.weakly_connected_components(G), key=len, reverse=True)
G0 = G.subgraph(Gcc[0])

In [9]:
nx.write_weighted_edgelist(G0, "test.weighted.edgelist.largest.component")

In [8]:
N = len(G)
L = G.size()
degrees = [G.degree(node) for node in G]
kmin = min(degrees)
kmax = max(degrees)

In [9]:
print("Number of nodes: ", N)
print("Number of edges: ", L)
print()
print("Average degree: ", 2*L/N)
print("Average degree (alternate calculation)", np.mean(degrees))
print()
print("Minimum degree: ", kmin)
print("Maximum degree: ", kmax)

Number of nodes:  8979
Number of edges:  14040

Average degree:  3.1272970263949214
Average degree (alternate calculation) 3.1272970263949214

Minimum degree:  1
Maximum degree:  381


In [14]:
avg_path_length = nx.average_shortest_path_length(G)

NetworkXError: Graph is not weakly connected.

In [15]:
G_undirected = nx.read_edgelist('../networkx_df.csv', comments='#',
                     create_using=nx.Graph(), 
                     delimiter=',', 
                     nodetype=int, 
                     encoding='utf-8')

In [16]:
avg_path_length = nx.average_shortest_path_length(G_undirected)

NetworkXError: Graph is not connected.

In [12]:
from networkx.generators.degree_seq import expected_degree_graph


DP_G = expected_degree_graph([deg for (_, deg) in G.degree()], selfloops=False)

In [13]:
N_DP = len(DP_G)
L_DP = DP_G.size()
degrees_dp = [DP_G.degree(node) for node in DP_G]
kmin_dp = min(degrees_dp)
kmax_dp = max(degrees_dp)

print("Number of nodes: ", N_DP)
print("Number of edges: ", L_DP)
print()
print("Average degree: ", 2*L_DP/N_DP)
print("Average degree (alternate calculation)", np.mean(degrees_dp))
print()
print("Minimum degree: ", kmin_dp)
print("Maximum degree: ", kmax_dp)

Number of nodes:  8979
Number of edges:  13843

Average degree:  3.083416861565876
Average degree (alternate calculation) 3.083416861565876

Minimum degree:  0
Maximum degree:  346


## Do This 1000 times and compare

In [29]:
clustering_coefs= []
transitivity_metrics = []

for i in range(1000):    
    DP_G = expected_degree_graph([deg for (_, deg) in G.degree()], selfloops=True)
    degrees_dp = [DP_G.degree(node) for node in DP_G]
    avg_k = np.mean(degrees_dp)

    #Then run the calculate metrics here 
    transitivity_metrics.append(nx.transitivity(DP_G))
    clustering_coefs.append(nx.average_clustering(DP_G))
    # Persist the node degrees to a csv
    degrees = [DP_G.degree(node) for node in DP_G]
    clustering = [nx.clustering(DP_G, node) for node in DP_G]

    with open('dp_degrees_1.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(degrees) 
            
    with open('dp_clustering_1.csv', mode='a', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(clustering) 
                

                

In [30]:
c  = pd.DataFrame(clustering_coefs)
t = pd.DataFrame(transitivity_metrics)

c.to_csv("dp_clustering.csv")
t.to_csv("dp_transitivity.csv")

In [31]:
#load the Data in from Pandas and average it
c_df = pd.read_csv("dp_clustering.csv")
t_df = pd.read_csv("dp_transitivity.csv")

In [32]:
c_df.head()

,Unnamed: 0,0
0,0,0.044221
1,1,0.041587
2,2,0.045878
3,3,0.043583
4,4,0.045137


In [33]:
c_df["0"].mean()

0.04316290458763149

In [34]:
t_df["0"].mean()

0.09435404013750476

In [35]:
#Actual Graph Transitivity
nx.transitivity(G)

0.020056849493865474

In [36]:
#Actual Graph Cluster Coefficient 
nx.average_clustering(G)

0.009245250908573041

In [37]:
#Actual Graph Cluster Coefficient 
nx.average_clustering(G_undirected)

0.016744035180022453

In [38]:
#Actual Graph Transitivity
nx.transitivity(G_undirected)

0.004707779939921785